In [95]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

In [96]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [97]:
from sklearn.preprocessing import StandardScaler

In [98]:
housing_data_plus_bias_scaled = StandardScaler().fit_transform(housing_data_plus_bias)

In [99]:
n_epochs = 1000
learning_rate = 0.01

In [106]:
tf.reset_default_graph()

In [107]:
class LR_Model:
    def __init__(self, mode):
        self._build_graph(mode)
    
    def _build_graph(self, mode='train'):
        self.X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
        self.y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

        theta = tf.get_variable(name='theta', shape=(n+1, 1), dtype=tf.float32)

        y_pred = tf.matmul(X, theta, name='predictions')
        error = y_pred - y
        if mode == 'train':
            mse = tf.reduce_mean(tf.square(error), name='mse')
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
            training_op = optimizer.minimize(mse)
            tf.summary.scalar("cost_function", mse)
            self.train_op = training_op
        else:
            
            loss = tf.reduce_mean(error, name='loss')
            self.loss = loss
            
        self.merged = tf.summary.merge_all()
        saver = tf.train.Saver()
        
        self.saver = saver
            
    def close_session(self):
        if self._session:
            self._session.close()
            

In [108]:
X = housing_data_plus_bias_scaled.astype(np.float32)
y = housing.target.reshape(-1, 1)

In [109]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = housing_data_plus_bias_scaled[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [110]:
batch_size = 100
n_epochs = 10
n_batches = int(np.ceil(m / batch_size))

In [111]:
with tf.variable_scope('root'):
    train_model = LR_Model('train')
    initializer = tf.global_variables_initializer()

with tf.variable_scope('root', reuse=True):
    eval_model = LR_Model('eval')

with tf.Session() as sess:
    sess.run(initializer)
    train_writer = tf.summary.FileWriter('/Users/luoshixin/Downloads/graph', sess.graph)
    train_writer.add_graph(tf.get_default_graph())
    i = 0
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            i += 1
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            _, summary = sess.run([train_model.train_op, train_model.merged], 
                              feed_dict={train_model.X: X_batch, train_model.y: y_batch})
#             train_writer.add_summary(summary, i)
            
        sess.run([eval_model.loss], feed_dict={eval_model.X: X_batch, eval_model.y: y_batch})

